In [143]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

In [163]:
#Settings
%matplotlib inline
pd.set_option('display.max_columns', 97)

def read_features():
    
    raw_data = pd.read_csv("2016-SFO-Customer-Survey-Data.csv")
    
    data = raw_data.drop(["*RESPNUM","CCGID", "RUNID", "DAY", "INTDATE", "EGYPTAIR", "SAQ", "DEST CODE", 
                         "AIRLINE CODE", "STRATA", "PEAK", "GATE", "METHOD", "FLIGHT", "LANG", "WEIGHT", 
                         "FLIGHT", "DESTGEO","Q3GETTO1", "Q3GETTO2", "Q3GETTO3", "Q3PARK", "Q8COM", "Q8COM2", 
                         "Q8COM3", "Q8COM4", "Q8COM5", "Q9COM", "Q9COM2", "Q9COM3", "Q10COM1", "Q10COM2", 
                         "Q10COM3", "Q12PRECOM1", "Q12PRECOM2", "Q12PRECOM3", "Q15COM1", "Q15COM2", "Q15COM3", 
                         "ARRTIME", "DEPTIME", "HOME"], axis = 1)
    
    return data

def prep_missing(data):
    
    # Survey codes blank/multiple responses as zeros. Replace all 0s and "0"s with NaN. 
    data = data.replace(0, np.nan)
    data = data.replace("0", np.nan)
    
    # Calculate threshold based on percentage of columns
    #TODO - Missing Data: remove hardcoding of threshold level
    threshold = len(data.index) * .15
    
    # Find columns with missing data above a certain threshold
    high_missing = data.columns[data.isnull().sum() > threshold].tolist()
    
    # Keep only columns with low missing percentage from data
    imputables = data.drop(high_missing, axis=1)
    
    # Rename column names to human-readable text
    imputables_renamed = imputables.rename(columns={'BAREA': 'Trav_Boarding_area', 'AIRLINE': 'Trav_Airline', 
                                                    'DESTINATION': 'Trav_Destination', 
                                                    'DESTMARK': 'Trav_Dest_market_size', 
                                                    'HOWLONG': 'Trav_Arr_to_dep_time',
                                                    'Q2PURP1': 'Trav_Travel_purpose', 
                                                    'Q4BAGS': 'Has_checked_baggage','Q4STORE': 'Shopped_stores', 
                                                    'Q4FOOD': 'Shopped_restaurant', 'Q4WIFI': 'Used_free_wifi',
                                                    'Q5TIMESFLOWN': 'Trav_Times_flownSFO', 
                                                    'Q5FIRSTTIME': 'Trav_Is_first_SFO',
                                                    'Q6LONGUSE': 'Trav_yearsUsed_group', 'Q7ART': 'Rate_art', 
                                                    'Q7FOOD': 'Rate_food', 'Q7STORE': 'Rate_stores', 
                                                    'Q7SIGN': 'Rate_signs', 'Q7WALKWAYS': 'Rate_moving_walkways', 
                                                    'Q7SCREENS': 'Rate_infoscreens', 
                                                    'Q7INFODOWN': 'Rate_infobooth_LL', 
                                                    'Q7INFOUP': 'Rate_infobooth_UL', 'Q7WIFI': 'Rate_free_wifi', 
                                                    'Q7ROADS': 'Rate_roadsigns', 'Q7PARK': 'Rate_parking', 
                                                    'Q7AIRTRAIN': 'Rate_airtrain', 
                                                    'Q7LTPARKING': 'Rate_parking_shuttle', 
                                                    'Q7RENTAL': 'Rate_rentacar', 'Q7ALL': 'Rate_SFO_overall', 
                                                    'Q9BOARDING': 'Clean_boarding_area', 
                                                    'Q9AIRTRAIN': 'Clean_airtrain', 
                                                    'Q9RENTAL': 'Clean_rentacar', 'Q9FOOD': 'Clean_restaurant', 
                                                    'Q9RESTROOM': 'Clean_restoom', 'Q9ALL': 'Clean_overall', 
                                                    'Q10SAFE': 'How_safe_overall', 'Q11TSAPRE': 'Used_TSA_Precheck', 
                                                    'Q13 COUNTY': 'Trav_County_of_origin', 
                                                    'Q13GETRATE': 'Rate_trip_to_airport', 
                                                    'Q14FIND': 'Ease_of_navigating_inside', 
                                                    'Q14PASSTHRU': 'Ease_of_passing_security', 
                                                    'Q15PROBLEM': 'Encountered_problems', 'Q16LIVE': 'Trav_How_local', 
                                                    'Q17CITY': 'Trav_City', 'Q17COUNTRY': 'Trav_Country', 
                                                    'Q18PET': 'Trav_with_pet', 'Q19AGE': 'Trav_age_group', 
                                                    'Q20GENDER': 'Trav_gender', 'Q22FLY': 'Is_Frequent_flyer', 
                                                    'Q23SJC': 'Used_SJC', 'Q23OAK': 'Used_OAK'
                                                    })
    return imputables_renamed

def encode_categories(data):
    
    label_encoder = LabelEncoder()
    encoded_categories = data.apply(label_encoder.fit_transform)
    return encoded_categories
 
def get_categoricals(data):
    
    categorical_columns = ['Trav_Boarding_area', 'Trav_Airline', 'Trav_Destination', 'Trav_Dest_market_size',
                           'Trav_Travel_purpose', 'Has_checked_baggage', 'Shopped_stores', 'Shopped_restaurant',
                           'Trav_Times_flownSFO', 'Trav_Is_first_SFO', 'Trav_yearsUsed_group', 'Used_TSA_Precheck',
                           'Trav_County_of_origin', 'Encountered_problems', 'Trav_How_local','Trav_City', 
                           'Trav_Country', 'Trav_with_pet', 'Trav_age_group', 'Trav_gender', 'Is_Frequent_flyer',
                           'Used_SJC','Used_OAK'
                          ]
    
    categorical_features = data[categorical_columns]
         
    #Convert to categories dtype   
    for col in categorical_columns:
        categorical_features[col] = categorical_features[col].astype('category')
        
    #Remove erroneous responses
    categorical_features['Trav_age_group'] = categorical_features['Trav_age_group'].cat.remove_categories(['Under 18','Under 19', 'Under 20', 'Under 21', 'Under 22',
                                                      'Under 23', 'Under 24', 'Under 25', 'Under 26', 'Under 27',
                                                'Under 28', 'Under 29', 'Under 30', 'Under 31', 'Under 32'])  
    
    encoded_cats = encode_categories(categorical_features)
    
    # TODO: Combine times flown and sfo (maybe yearsUsed?)
    return encoded_cats

def get_numerics(data, cols_to_remove):
    
    numeric_features = data.drop(cols_to_remove, axis=1)
    return numeric_features

def get_xy_vars(data, col_name):
    
    y = data[col_name]
    x = data.drop(col_name, axis=1)
    return y, x



if __name__ == "__main__":
    data=read_features()
    prepped = prep_missing(data)
    cats = get_categoricals(prepped)
    nums = get_numerics(prepped, cats)
    y_all, x_nums = get_xy_vars(nums, 'Rate_SFO_overall')